In [34]:
# import json
import pandas as pd
# import os
from nltk.corpus import stopwords
import nltk
import pymorphy2
import re
import string
from tqdm import tqdm
# from tqdm.notebook import tqdm as notebook_tqdm
from collections import Counter
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.decomposition import LatentDirichletAllocation
import numpy as np
# import matplotlib.pyplot as plt
from gensim.models.ldamulticore import LdaMulticore
# from gensim.models import *
from gensim import corpora
from datetime import datetime

tqdm.pandas()
nltk.download('stopwords')

import warnings
warnings.filterwarnings('ignore')

from multiprocessing import Pool
morph = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mchelushkin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
commentsDF = pd.read_csv('commentsRAW.csv', usecols=['id', 'date', 'text'])
answersDF = pd.read_csv('answersRAW.csv', usecols=['id', 'date', 'text'])
commentsDF = commentsDF[commentsDF.text.notna()]
answersDF = answersDF[answersDF.text.notna()]

In [4]:
allCommentsDF = commentsDF.append(answersDF, ignore_index=True)
allCommentsDF.shape

(805421, 3)

In [9]:
# вк собака объединяет символы - ковычки и троеточия
import emoji
def deEmojify(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r' ', text)

def give_emoji_free_text(text):
    return emoji.get_emoji_regexp().sub(r'', text)

punctuation = string.punctuation + '…«»—–'
def clean_text(text):
# Make text lowercase
    text = text.lower()
# remove text in square brackets
    text = re.sub(r'\[.*?\]', ' ', text)
# remove urls
    text = re.sub(r"http\S+", ' ', text)
# remove emojies
    text = give_emoji_free_text(text)
# remove punctuation   
    text = re.sub(r'[%s]' % re.escape(punctuation), ' ', text)
# remove numbers
    text = re.sub(r'[0-9]', ' ', text)
# remove non letters
    text = re.sub(r'[^\w\s]', '', text)
# remove dublicate spaces
    text = re.sub('\s\s+', ' ', text)
    return text
comments_df_clean = pd.DataFrame(allCommentsDF.text.progress_apply(lambda x: clean_text(x)))

100%|██████████| 805421/805421 [07:09<00:00, 1873.74it/s]


In [17]:
%%time

stopwords_list = stopwords.words('russian')
stopwords_list.append('это')
stopwords_list.append('всё')
stopwords_list.append('ещё')
stopwords_list.append('весь')
stopwords_list.append('человек')
stopwords_list.append('свой')
stopwords_list.append('который')
stopwords_list.append('мочь')
stopwords_list.append('')

def lemmatize(text, lemmer = morph, stopwords = stopwords_list):
    words = text.split(' ')
    lemmas = [lemmer.parse(w)[0].normal_form for w in words]
    return [w for w in lemmas if not w in stopwords]

with Pool(processes = 4) as pool:
    tmp = pool.starmap(lemmatize, zip(comments_df_clean.text))

CPU times: user 6.03 s, sys: 1.14 s, total: 7.16 s
Wall time: 33min 39s


In [18]:
comments_lemmatized_df = pd.DataFrame({'text' : tmp, 'date' : allCommentsDF.date})

In [19]:
def wordsCount(texts):
    count = Counter()
    for words in texts:
        for word in words:
            count[word] += 1
    return count

def removeNotCommonWords(text, word_counter):
    return [word for word in text if word_counter[word] > 5]


def func(df):
    word_counter = wordsCount(df.text.values.tolist())
    
    df.text = df.text.apply(lambda x: removeNotCommonWords(x, word_counter))
    df.text = df.text.apply(lambda x: ' '.join(x))
    df.text = df.text.apply(lambda x: x.split())
    df = df[df.text.apply(lambda x: len(x) >= 20 and len(x) <= 300)]
    return df

In [22]:
clean_texts = func(comments_lemmatized_df)

In [24]:
dictionary = corpora.Dictionary(clean_texts.text)

dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n=None) # игнорируем слова, которые встречаются реже 10 раз или составляют более 0.9 словаря 
dictionary.save('lda.dict')

In [25]:
corpus = [dictionary.doc2bow(text) for text in clean_texts.text]
corpora.MmCorpus.serialize('lda_gensim.model', corpus)

In [27]:
%%time

lda = LdaMulticore(corpus=corpus, id2word=dictionary, num_topics=15, passes = 5, iterations = 60)

CPU times: user 2min 18s, sys: 5.45 s, total: 2min 23s
Wall time: 2min 59s


In [28]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

%time vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

/home/mchelushkin/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_lda.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps
/home/mchelushkin/anaconda3/lib/python3.8/site-packages/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/home/mchelushkin/anaconda3/lib/python3.8/site-packages/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))


CPU times: user 33.9 s, sys: 477 ms, total: 34.4 s
Wall time: 37.6 s


In [30]:
topic_weights = []
topics = []
for row_list in tqdm(lda[corpus]):
    topic_weights.append([w for i, w in row_list])
    topics.append([i for i, w in row_list])

/home/mchelushkin/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 90171/90171 [00:48<00:00, 1874.90it/s]


In [35]:
topic_labels = list(range(1, 16))

clean_texts['topic_weights'] = topic_weights
clean_texts['topics'] = topics
clean_texts['best_topic'] = clean_texts.topic_weights.apply(lambda x: np.argmax(x))
clean_texts['formated_date'] = clean_texts.date.apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S')[:10])
best_topic_weights = []
best_topic_label = []
for row in clean_texts.iterrows():
    best_topic_weights.append(row[1]['topic_weights'][row[1].best_topic])
    best_topic_label.append(topic_labels[row[1]['topics'][row[1].best_topic]])

clean_texts['best_topic_weights'] = best_topic_weights
clean_texts['best_topic_label'] = best_topic_label

In [36]:
clean_texts[['date', 'formated_date', 'best_topic_weights', 'best_topic_label']].to_csv('data_for_visualization.csv', index_label='id')